# Read in in LA-COMPASS 2D data

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

from pylacompass import read_data, read_hdf5_file

%matplotlib inline
plt.style.use({'figure.dpi': 150})

## Reading data

Set filename of the `hdf5` file

In [ ]:
fname = 'data.hdf5'

This reads in the specified directory from a sub-directory `bin_data`. The simulation parameters are parsed from the file given as `inputfile`. If keyword `fname` is given, we store the result in a hdf5 file with that name.

In [ ]:
d = read_data(directory='.', inputfile='planet2D.input.new', n=-1, fname=fname)

Once the hdf5 file exists, we can also read from it. By default that just opens the file and the data is read when needed. If everything is to be read at once into memory, pass the `lowmem=False` keyword.

In [ ]:
d, f5  = read_hdf5_file(fname, n=-1, lowmem=True)

## 2D plot

In [ ]:
f, ax = plt.subplots(figsize=(6, 5))

cc = ax.pcolormesh(d.xy1, d.xy2, np.log10(d.sigma_d[:, :, 10].T + 1e-45), rasterized=True)
plt.colorbar(cc)
ax.set_aspect(1)

axins = zoomed_inset_axes(ax, 4, loc=2, bbox_to_anchor=(0.2, 0.8), bbox_transform=ax.figure.transFigure)
axins.pcolormesh(d.xy1, d.xy2, np.log10(d.sigma_d[:, :, 10].T + 1e-45), rasterized=True)
axins.set_aspect(1)
axins.set_xlim(1.1, 1.6)
axins.set_ylim(0.0, 0.5)

axins.axes.get_xaxis().set_visible(False)
axins.axes.get_yaxis().set_visible(False)

mark_inset(ax, axins, loc1=1, loc2=3, fc='none', ec='0.5')

f.savefig('xy.pdf')

### Plot 1D (azimuthally averaged) gas and total dust

In [ ]:
f, ax = plt.subplots()
ax.loglog(d.x, d.sigma_g[:, :].mean(1), label='gas')
ax.loglog(d.x, d.sigma_d[:, :, :].sum(-1).mean(-1), label='dust')
ax.set_xlabel('radius')
ax.set_ylabel('$\Sigma$ [g cm$^{-2}$]')
ax.set_ylim(1e-4, 1e3)
f.savefig('sigma_avg.pdf')

## Plot azimuthally averaged size distribution

define the particle size array

In [ ]:
a = np.logspace(
    np.log10(d['params']['size_of_dust']),
    np.log10(d['params']['size_of_dust_mx']),
    d['params']['nDust']
    )

r  = 1.3
ir = d.x[()].searchsorted(r)

f, ax = plt.subplots()
ax.loglog(a, d.sigma_d[ir, :, :].mean(0))
ax.set_xlabel('particle size [cm]')
ax.set_ylabel('$\sigma$ [g cm$^{-2}$]')
ax.set_ylim(1e-10, 1e-1)
f.savefig('size_at_{:.2g}.pdf'.format(r))

Close the hdf5-file

In [ ]:
f5.close()